# Credit Card Default Prediction - ML Pipeline

This notebook contains the complete machine learning pipeline for predicting credit card defaults.

## Imports

In [10]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import shap
import warnings
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
	roc_auc_score, precision_score, recall_score, f1_score,
	confusion_matrix, average_precision_score, brier_score_loss, accuracy_score, classification_report
)
from sklearn.inspection import permutation_importance
from sklearn.base import clone
from scipy import stats
from scipy.stats import zscore
from scipy.stats.mstats import winsorize
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')

## Data Preprocessing Functions

In [11]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from scipy.stats.mstats import winsorize
from imblearn.over_sampling import SMOTE

def preprocess_data(file_path, verbose=False, apply_scaling=False, split_data=False, apply_smote=False, test_size=0.2, random_state=42):
	"""
	Preprocess the credit card default dataset.
	
	Args:
		file_path: Path to the Excel file containing the dataset
		verbose: If True, print progress information. If False, no output.
		apply_scaling: If True, apply robust scaling (robust if anomalies present, else standard)
		split_data: If True, return train/test split. If False, return single DataFrame
		apply_smote: If True, apply SMOTE to training data (only works if split_data=True)
		test_size: Proportion of dataset to use for testing (default 0.2)
		random_state: Random seed for reproducibility
	
	Returns:
		If split_data=False: Preprocessed pandas DataFrame
		If split_data=True: (X_train, X_test, y_train, y_test, scaler) tuple
			scaler will be None if apply_scaling=False, otherwise the fitted scaler object
	"""
	# Helper function for conditional printing
	def log(*args, **kwargs):
		if verbose:
			print(*args, **kwargs)
	
	# Load the dataset
	df = pd.read_excel(file_path)
	
	# Rename columns to their descriptive names
	column_mapping = {
		'X1': 'LIMIT_BAL',
		'X2': 'SEX',
		'X3': 'EDUCATION',
		'X4': 'MARRIAGE',
		'X5': 'AGE',
		'X6': 'PAY_0',
		'X7': 'PAY_2',
		'X8': 'PAY_3',
		'X9': 'PAY_4',
		'X10': 'PAY_5',
		'X11': 'PAY_6',
		'X12': 'BILL_AMT1',
		'X13': 'BILL_AMT2',
		'X14': 'BILL_AMT3',
		'X15': 'BILL_AMT4',
		'X16': 'BILL_AMT5',
		'X17': 'BILL_AMT6',
		'X18': 'PAY_AMT1',
		'X19': 'PAY_AMT2',
		'X20': 'PAY_AMT3',
		'X21': 'PAY_AMT4',
		'X22': 'PAY_AMT5',
		'X23': 'PAY_AMT6',
		'Y': 'default_payment_next_month'
	}
	
	# Filter mapping to only include columns that exist, then rename all at once
	# (Learned this trick on leetcode)
	filtered_mapping = {old: new for old, new in column_mapping.items() if old in df.columns}
	if filtered_mapping:
		df = df.rename(columns=filtered_mapping)
		log("Renamed columns:")
		for old_name, new_name in filtered_mapping.items():
			log(f"  {old_name} -> {new_name}")
		log()
	
	"""
	Load and check the dataset
	"""
	
	log("INITIAL DATA EXPLORATION")
	
	log(f"\nDataFrame head:\n{df.head()}")
	
	
	log(f"\nDataFrame tail:\n{df.tail()}")
	
	log(f"\nDataFrame info:\n{df.info()}")
	
	log(f"\nDataFrame describe:\n{df.describe()}")
	
	log(f"\nDataFrame shape: {df.shape[0]} rows, {df.shape[1]} columns")
	log(f"Expected size: ~30,000 rows")
	if df.shape[0] < 2500: # Minimum requirement of 2,500 rows
		log("WARNING: Dataset size below minimum requirement of 2,500 rows")
	elif 25000 <= df.shape[0] <= 35000:
		log("Dataset size looks good")
	else:
		log(f"Dataset size: {df.shape[0]} rows (expected ~30,000)")
	
	log(f"\nDataFrame columns:\n{df.columns.tolist()}")
	log(df.columns.tolist())
	
	log(f"\nDataFrame dtypes:\n{df.dtypes}")
	
	log(f"\nMissing values count:\n{df.isna().sum()}")
	missing_counts = df.isna().sum()
	log(missing_counts[missing_counts > 0])
	
	log(f"\nDuplicate rows:\n{df.duplicated().sum()}")
	
	"""
	Remove unnecessary columns
	"""
	log("REMOVING UNNECESSARY COLUMNS")
	
	# Remove ID/index columns - they're not useful for prediction
	# Common names: ID, id, Unnamed: 0, index (default for excel files)
	cols_to_drop = []
	for col in df.columns:
		col_lower = str(col).lower()
		if col_lower in ['id', 'unnamed: 0', 'index'] or 'id' in col_lower:
			cols_to_drop.append(col)
	
	if cols_to_drop:
		log(f"Dropping columns: {cols_to_drop}")
		df = df.drop(columns=cols_to_drop)
	else:
		log("No obvious index columns found to drop")
	
	log("HANDLING MISSING VALUES")
	
	missing_counts = df.isna().sum()
	missing_cols = missing_counts[missing_counts > 0]
	
	if len(missing_cols) == 0:
		log("No missing values found")
	else:
		log(f"Columns with missing values: {len(missing_cols)}")
		
		# First pass: drop columns with too many missing values (10% or more)
		cols_to_drop_missing = []
		for col in missing_cols.index:  
			missing_pct = (missing_counts[col] / len(df)) * 100
			if missing_pct >= 10:
				log(f"{col}: {missing_counts[col]} missing ({missing_pct:.2f}%) - Dropping column")
				cols_to_drop_missing.append(col)
		
		if cols_to_drop_missing:
			log(f"\nDropping columns with too many missing values: {cols_to_drop_missing}")
			df = df.drop(columns=cols_to_drop_missing)
			missing_counts = df.isna().sum()
			missing_cols = missing_counts[missing_counts > 0]
		
		# Check if we need KNN imputation (for numeric columns with >=10% missing)
		numeric_cols_for_knn = []
		for col in missing_cols.index:
			if df[col].dtype in ['int64', 'float64']:
				missing_pct = (missing_counts[col] / len(df)) * 100
				if missing_pct >= 10:
					numeric_cols_for_knn.append(col)
		
		# Handle KNN imputation for all numeric columns at once if needed
		if numeric_cols_for_knn:
			log(f"\nUsing K-NN imputation for numeric columns with >=10% missing: {numeric_cols_for_knn}")
			imputer = KNNImputer(n_neighbors=5)
			numeric_cols_all = df.select_dtypes(include=[np.number]).columns
			df_numeric = df[numeric_cols_all].copy()
			imputed = imputer.fit_transform(df_numeric)
			df[numeric_cols_all] = imputed
			# Update missing counts after KNN
			missing_counts = df.isna().sum()
			missing_cols = missing_counts[missing_counts > 0]
		
		# Handle remaining missing values column by column
		for col in missing_cols.index:
			missing_pct = (missing_counts[col] / len(df)) * 100
			log(f"\n{col}: {missing_counts[col]} missing ({missing_pct:.2f}%)")
			
			# If insignificant count, just drop the rows
			if missing_pct < 1:
				log(f"  -> Dropping {missing_counts[col]} rows with missing values")
				df = df.dropna(subset=[col])
			else:
				# Handle based on data type
				if df[col].dtype in ['int64', 'float64']:
					# Numeric: use mean (KNN already handled if needed)
					mean_val = df[col].mean()
					log(f"  -> Filling {missing_counts[col]} missing values with mean: {mean_val:.2f}")
					df[col] = df[col].fillna(mean_val)
				else:
					# Categorical: impute with mode
					mode_val = df[col].mode()[0] if len(df[col].mode()) > 0 else 'Unknown'
					log(f"  -> Filling {missing_counts[col]} missing values with mode: {mode_val}")
					df[col] = df[col].fillna(mode_val)
	
	log(f"\nShape after handling missing values: {df.shape}")
	
	"""
	Correct types and encoding
	"""
	log("CORRECTING TYPES AND ENCODING")
	
	# Float -> Int (where values are whole numbers)
	for col in df.select_dtypes(include=['float64']).columns:
		if df[col].notna().all():
			if (df[col] % 1 == 0).all():
				log(f"Converting {col} from float to int")
				df[col] = df[col].astype('int64')
	
	# String -> Int (where applicable - e.g., "1", "2" -> 1, 2)
	string_to_int_count = 0
	for col in df.select_dtypes(include=['object']).columns:
		try:
			# Try converting to numeric
			converted = pd.to_numeric(df[col], errors='coerce')
			if converted.notna().sum() / len(df) > 0.9:  # If 90%+ can be converted
				converted_count = converted.notna().sum()
				log(f"Converting {col} from string to int ({converted_count} values converted)")
				df[col] = converted.fillna(0).astype('int64')
				string_to_int_count += 1
		except:
			pass
	
	if string_to_int_count > 0:
		log(f"Total columns converted from string to int: {string_to_int_count}")
	
	# Identify categorical columns for one-hot encoding
	# Look for columns with limited unique values (nominal categories)
	categorical_cols = []
	for col in df.columns:
		if df[col].dtype == 'object' or df[col].dtype.name == 'category':
			categorical_cols.append(col)
		elif df[col].dtype in ['int64', 'int32']:
			# Could be categorical if low cardinality
			unique_count = df[col].nunique()
			if unique_count <= 10 and unique_count < len(df) * 0.1:
				# Low cardinality, might be categorical
				# But we'll handle payment status separately
				pass
	
	# One-hot encode nominal categories (where order doesn't matter)
	# Common ones: SEX, EDUCATION, MARRIAGE
	nominal_cols = []
	for col in categorical_cols:
		col_lower = str(col).lower()
		if any(x in col_lower for x in ['sex', 'gender', 'education', 'marriage', 'marital']):
			nominal_cols.append(col)
	
	# Also check for other low-cardinality int columns that might be nominal
	for col in df.select_dtypes(include=['int64', 'int32']).columns:
		col_lower = str(col).lower()
		if col_lower in ['sex', 'education', 'marriage'] and df[col].nunique() <= 10:
			nominal_cols.append(col)
	
	log(f"\nOne-hot encoding nominal categories: {nominal_cols}")
	for col in nominal_cols:
		if col in df.columns:
			dummies = pd.get_dummies(df[col], prefix=col, drop_first=True)
			df = pd.concat([df, dummies], axis=1)
			df = df.drop(columns=[col])
			log(f"  -> Encoded {col} into {len(dummies.columns)} binary columns")
	
	# Ordinal encode payment status (where order matters)
	# Look for PAY_ columns or payment status columns
	payment_cols = []
	for col in df.columns:
		col_lower = str(col).lower()
		if 'pay' in col_lower and ('status' in col_lower or col_lower.startswith('pay_')):
			payment_cols.append(col)
	
	log(f"\nPayment status columns (keeping as ordinal): {payment_cols}")
	# Payment status is already numeric typically, but we'll ensure it's int
	for col in payment_cols:
		if col in df.columns:
			df[col] = df[col].astype('int64')
			log(f"  -> {col} kept as ordinal (int)")
	
	# Int -> Category (where applicable - for memory efficiency)
	# Only for very low cardinality columns that aren't being used in calculations
	for col in df.select_dtypes(include=['int64', 'int32']).columns:
		if col not in payment_cols and df[col].nunique() <= 5:
			col_lower = str(col).lower()
			# Don't convert if it's a bill amount, payment amount, or limit
			if not any(x in col_lower for x in ['bill', 'pay_amt', 'limit', 'bal']):
				log(f"Converting {col} to category for memory efficiency")
				df[col] = df[col].astype('category')
	
	# Date(ish) -> Date-time
	# Look for date-like columns
	for col in df.select_dtypes(include=['object']).columns:
		try:
			# Try parsing as date
			parsed = pd.to_datetime(df[col], errors='coerce')
			if parsed.notna().sum() / len(df) > 0.5:  # If 50%+ can be parsed
				log(f"Converting {col} to datetime")
				df[col] = parsed
		except:
			pass
	
	log(f"\nShape after type corrections: {df.shape}")
	
	"""
	Anomaly detection
	"""
	
	log("REMOVING OUTLIERS")
	
	initial_rows = len(df)
	
	# Method 1: 2 Standard Deviations (2-SD)
	log("Using 2-SD method:")
	numeric_cols = df.select_dtypes(include=[np.number]).columns
	outliers_2sd = set()
	
	for col in numeric_cols:
		z_scores = np.abs(zscore(df[col].dropna()))
		outlier_mask = z_scores > 2
		outlier_count = outlier_mask.sum()
		if outlier_count > 0:
			outlier_indices = df[col].index[outlier_mask]
			outliers_2sd.update(outlier_indices)
			log(f"  {col}: {outlier_count} outliers (|z| > 2)")
	
	# Method 2: IQR (Interquartile Range)
	log("Using IQR method:")
	outliers_iqr = set()
	
	for col in numeric_cols:
		Q1 = df[col].quantile(0.25)
		Q3 = df[col].quantile(0.75)
		IQR = Q3 - Q1
		lower_bound = Q1 - 1.5 * IQR
		upper_bound = Q3 + 1.5 * IQR
		outlier_mask = (df[col] < lower_bound) | (df[col] > upper_bound)
		outlier_count = outlier_mask.sum()
		if outlier_count > 0:
			outlier_indices = df[col].index[outlier_mask]
			outliers_iqr.update(outlier_indices)
			log(f"  {col}: {outlier_count} outliers (outside IQR bounds)")
	
	# Combine both methods - remove if flagged by either
	all_outliers = outliers_2sd.union(outliers_iqr)
	log(f"\nTotal unique outlier rows: {len(all_outliers)}")
	
	# For now, we'll use winsorization instead of removal to preserve data
	# Winsorize extreme values to 1st and 99th percentiles
	log("\nWinsorizing extreme values (1st and 99th percentiles):")
	total_winsorized = 0
	for col in numeric_cols:
		# Skip if it's a binary/categorical column
		if df[col].nunique() <= 2:
			continue
		
		original_values = df[col].copy()
		original_min = df[col].min()
		original_max = df[col].max()
		
		# Winsorize at 1st and 99th percentiles
		winsorized = winsorize(df[col].values, limits=[0.01, 0.01])
		# Convert masked array to regular array
		df[col] = np.array(winsorized)
		
		# Count how many values were actually changed
		values_changed = (original_values != df[col]).sum()
		if values_changed > 0:
			total_winsorized += values_changed
			log(f"  {col}: {values_changed} values clipped (min: {original_min:.2f}->{df[col].min():.2f}, max: {original_max:.2f}->{df[col].max():.2f})")
	
	log(f"\nTotal values winsorized across all columns: {total_winsorized}")
	
	# Group rare values into "Other" for categorical columns
	log("\nGrouping rare categorical values into 'Other':")
	for col in df.select_dtypes(include=['object', 'category']).columns:
		value_counts = df[col].value_counts()
		# If a value appears in less than 1% of rows, group it as "Other"
		rare_threshold = len(df) * 0.01
		rare_values = value_counts[value_counts < rare_threshold].index.tolist()
		
		if len(rare_values) > 0 and len(rare_values) < len(value_counts):
			log(f"  {col}: grouping {len(rare_values)} rare values into 'Other'")
			df[col] = df[col].replace(rare_values, 'Other')
	
	log(f"\nShape after anomaly handling: {df.shape} (removed {initial_rows - len(df)} rows)")
	
	"""
	QOL FEATURES (Quality of Life / Derived Features)
	"""
	log("CREATING DERIVED FEATURES")
	
	# Find bill amount columns (BILL_AMT1, BILL_AMT2, etc.) - bill_amt is the default column name for bill amounts
	bill_cols = [col for col in df.columns if 'bill_amt' in str(col).lower()]
	# Also check for PAY_AMT columns (payment amounts) - pay_amt is the default column name for payment amounts
	pay_amt_cols = [col for col in df.columns if 'pay_amt' in str(col).lower()]
	
	log(f"Found bill columns: {bill_cols}")
	log(f"Found payment amount columns: {pay_amt_cols}")
	
	# Total bills paid to date (sum of all payment amounts)
	if pay_amt_cols:
		df['total_bills_paid_to_date'] = df[pay_amt_cols].sum(axis=1)
		log(f"Created: total_bills_paid_to_date")
	
	# Average bill amount
	if bill_cols:
		df['avg_bill'] = df[bill_cols].mean(axis=1)
		log(f"Created: avg_bill")
	
	# Total in processing (current outstanding bills)
	if bill_cols:
		df['current_outstanding'] = df[bill_cols].sum(axis=1)
		log(f"Created: current_outstanding")
	
	# Amortised debt (rough estimate: bills - payments over time)
	if bill_cols and pay_amt_cols:
		# Sum of bills minus sum of payments
		df['amortised_debt'] = df[bill_cols].sum(axis=1) - df[pay_amt_cols].sum(axis=1)
		# Can't have negative amortised debt in this context
		df['amortised_debt'] = df['amortised_debt'].clip(lower=0)
		log(f"Created: amortised_debt")
	
	# Temporal features - payment timing
	# Find payment status columns (PAY_0, PAY_2, etc. or PAY_1, PAY_2, etc.)
	pay_status_cols = [col for col in df.columns if 'pay' in str(col).lower() and col not in pay_amt_cols and 'status' not in str(col).lower()]
	# Filter to actual payment status columns (usually PAY_0, PAY_2, PAY_3, etc.)
	pay_status_cols = [col for col in pay_status_cols if any(x in col for x in ['PAY_', 'Pay_', 'pay_'])]
	
	if pay_status_cols:
		# How overdue on average (negative values = paid early, positive = overdue)
		# Payment status typically: -1 = paid duly, 0 = no consumption, 1+ = months overdue
		df['avg_overdue_months'] = df[pay_status_cols].mean(axis=1)
		log(f"Create: avg_overdue_months")
		
		# Max overdue months
		df['max_overdue_months'] = df[pay_status_cols].max(axis=1)
		log(f"Created: max_overdue_months")
		
		# Count of months with overdue payments
		df['months_overdue_count'] = (df[pay_status_cols] > 0).sum(axis=1)
		log(f"Created: months_overdue_count")
	
	# Credit utilisation
	# Look for limit and balance columns
	limit_col = None
	balance_col = None
	
	for col in df.columns:
		col_lower = str(col).lower()
		if 'limit' in col_lower and 'bal' in col_lower:
			limit_col = col
		if ('balance' in col_lower or 'bal' in col_lower) and 'limit' not in col_lower:
			balance_col = col
	
	if limit_col and balance_col:
		# Avoid division by zero
		df['credit_utilisation'] = np.where(
			df[limit_col] > 0,
			df[balance_col] / df[limit_col],
			0
		)
		# Cap at 1.0 (100% utilisation)
		df['credit_utilisation'] = df['credit_utilisation'].clip(upper=1.0)
		log(f"Created: credit_utilisation (using {limit_col} and {balance_col})")
	elif limit_col:
		# If we have limit but no current balance, use total bills as proxy
		if bill_cols:
			df['credit_utilisation'] = np.where(
				df[limit_col] > 0,
				df[bill_cols].sum(axis=1) / df[limit_col],
				0
			)
			df['credit_utilisation'] = df['credit_utilisation'].clip(upper=1.0)
			log(f"Created: credit_utilisation (using {limit_col} and bill amounts as proxy)")
	
	log(f"\nShaape after creating derived features: {df.shape}")
	
	"""
	Final summary
	"""
	log("FINAL DATA SUMMARY")
	
	log(f"\nFinal shape: {df.shape}")
	log(f"Final columns: {len(df.columns)}")
	log(f"\nColumn names:")
	for i, col in enumerate(df.columns, 1):
		log(f"  {i}. {col}")
	
	log(f"\nFinal dtypes:")
	log(df.dtypes.value_counts())
	
	log(f"\nFinal missing values:")
	final_missing = df.isna().sum()
	if final_missing.sum() > 0:
		log(final_missing[final_missing > 0])
	else:
		log("None")
	
	# Ensure cleaned dataset is still large enough
	log(f"\nDataset size validation:")
	log(f"Minimum requirement: 2,500 rows - Current: {df.shape[0]} rows")
	if df.shape[0] >= 2500:
		log("Dataset size meets minimum requirement")
	else:
		log("WARNING: Dataset size below minimum requirement")
		raise ValueError(f"Dataset too small: {df.shape[0]} rows (minimum: 2,500)")
	
	# Check class balance if target column exists
	target_col = 'default_payment_next_month'
	if target_col in df.columns:
		class_counts = df[target_col].value_counts()
		class_balance = class_counts.min() / class_counts.max()
		log(f"\nClass balance check:")
		log(f"  Class distribution:\n{class_counts}")
		log(f"  Balance ratio: {class_balance:.3f}")
		if class_balance < 0.25:  # Less than 25% means >75% in one class
			log(f"  WARNING: Class imbalance detected (>75% in majority class)")
			log(f"  Consider using class_weight='balanced' or SMOTE")
		else:
			log(f"  Class balance is acceptable")
	
	# Separate features and target if splitting
	if split_data:
		if target_col not in df.columns:
			raise ValueError("Cannot split data: target column 'default_payment_next_month' not found")
		
		scaler = None  # Initialize scaler variable
		X = df.drop(columns=[target_col])
		y = df[target_col]
		
		log("\nTRAIN/TEST SPLIT")
		
		# Stratified train/test split
		X_train, X_test, y_train, y_test = train_test_split(
			X, y, 
			test_size=test_size, 
			random_state=random_state,
			stratify=y  # Stratified split to maintain class distribution
		)
		
		log(f"Training set: {X_train.shape[0]} rows, {X_train.shape[1]} features")
		log(f"Test set: {X_test.shape[0]} rows, {X_test.shape[1]} features")
		log(f"Training class distribution:\n{y_train.value_counts()}")
		log(f"Test class distribution:\n{y_test.value_counts()}")
		
		# Apply SMOTE to training data if requested
		if apply_smote:
			log(f"\nApplying SMOTE to training data...")
			try:
				smote = SMOTE(random_state=random_state)
				X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
				log(f"Before SMOTE: {X_train.shape[0]} samples")
				log(f"After SMOTE: {X_train_resampled.shape[0]} samples")
				log(f"Resampled class distribution:\n{pd.Series(y_train_resampled).value_counts()}")
				X_train = pd.DataFrame(X_train_resampled, columns=X_train.columns, index=X_train.index[:len(X_train_resampled)])
				y_train = pd.Series(y_train_resampled, index=X_train.index)
			except Exception as e:
				log(f"WARNING: SMOTE failed: {e}")
				log("Continuing without SMOTE...")
		
		# Apply scaling if requested
		if apply_scaling:
			log("\nAPPLYING SCALING")
			
			# Use robust scaling if anomalies were detected (winsorization was applied)
			# Otherwise use standard scaling
			use_robust = total_winsorized > 0
			
			if use_robust:
				log("Using RobustScaler (anomalies detected in data)")
				scaler = RobustScaler()
			else:
				log("Using StandardScaler")
				scaler = StandardScaler()
			
			# Fit on training data, transform both train and test
			X_train_scaled = scaler.fit_transform(X_train)
			X_test_scaled = scaler.transform(X_test)
			
			# Convert back to DataFrame with original column names
			X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
			X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
			
			log("Scaling applied to training and test sets")
		
		log("\nFINAL SUMMARY")
		log(f"Training set shape: {X_train.shape}")
		log(f"Test set shape: {X_test.shape}")
		log(f"\nDataset ready for modeling!")
		
		# Always return scaler when splitting (None if scaling wasn't applied)
		return X_train, X_test, y_train, y_test, scaler
	
	# Apply scaling to entire dataset if requested (and not splitting)
	if apply_scaling:
		log("\nAPPLYING SCALING")
		
		# Use robust scaling if anomalies were detected (winsorization was applied)
		use_robust = total_winsorized > 0
		
		if use_robust:
			log("Using RobustScaler (anomalies detected in data)")
			scaler = RobustScaler()
		else:
			log("Using StandardScaler")
			scaler = StandardScaler()
		
		# Separate target if it exists
		if target_col in df.columns:
			X = df.drop(columns=[target_col])
			y = df[target_col]
			
			# Scale only features
			X_scaled = scaler.fit_transform(X)
			X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
			
			# Recombine
			df = pd.concat([X_scaled, y], axis=1)
		else:
			# Scale all numeric columns
			numeric_cols = df.select_dtypes(include=[np.number]).columns
			df_scaled = scaler.fit_transform(df[numeric_cols])
			df[numeric_cols] = df_scaled
		
		log("Scaling applied to dataset")
	
	log("\nFINAL SUMMARY")
	log(f"Final shape: {df.shape}")
	log(f"\nDataset ready for modeling!")
	
	return df

## Model Training Functions

In [12]:
import pandas as pd
import numpy as np
import time
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix, accuracy_score, average_precision_score
import warnings
warnings.filterwarnings('ignore')

def check_class_imbalance(y):
	"""
	Check if class imbalance is significant (>60:40 ratio).
	
	Returns:
		bool: True if imbalance is significant
	"""
	class_counts = y.value_counts()
	ratio = class_counts.min() / class_counts.max()
	return ratio < 0.4

def get_logistic_regression(class_imbalance=False, random_state=42):
	"""
	Create Logistic Regression model with L2 penalty.
	
	Args:
		class_imbalance: If True, use class_weight='balanced'
		random_state: Random seed
	
	Returns:
		LogisticRegression model
	"""
	if class_imbalance:
		return LogisticRegression(
			penalty='l2',
			solver='liblinear',
			class_weight='balanced',
			random_state=random_state,
			max_iter=1000
		)
	else:
		return LogisticRegression(
			penalty='l2',
			solver='saga',
			random_state=random_state,
			max_iter=1000
		)

def get_random_forest(random_state=42):
	"""
	Create Random Forest classifier as baseline.
	
	Returns:
		RandomForestClassifier model
	"""
	return RandomForestClassifier(
		n_estimators=100,
		random_state=random_state,
		n_jobs=-1
	)

def get_gradient_boosting(random_state=42):
	"""
	Create Gradient Boosting Tree classifier.
	
	Returns:
		GradientBoostingClassifier model
	"""
	return GradientBoostingClassifier(
		random_state=random_state,
		validation_fraction=0.1,
		n_iter_no_change=5
	)

def get_neural_network(random_state=42):
	"""
	Create simple Neural Network (MLP) for comparison.
	
	Returns:
		MLPClassifier model
	"""
	return MLPClassifier(
		hidden_layer_sizes=(100, 50),
		activation='relu',
		solver='adam',
		alpha=0.0001,
		batch_size='auto',
		learning_rate='constant',
		learning_rate_init=0.001,
		max_iter=500,
		random_state=random_state,
		early_stopping=True,
		validation_fraction=0.1
	)

def tune_model_randomized(model, param_distributions, X_train, y_train, 
						  n_iter=50, cv_fold=None, scoring='roc_auc', random_state=42, verbose=0):
	"""
	Use RandomizedSearchCV for broad hyperparameter search.
	
	Args:
		model: Base model to tune
		param_distributions: Dictionary of parameter distributions
		X_train: Training features
		y_train: Training target
		n_iter: Number of iterations
		cv_fold: Pre-defined CV fold object (for fairness across models)
		scoring: Scoring metric
		random_state: Random seed
		verbose: Verbosity level
	
	Returns:
		Best model from RandomizedSearchCV
	"""
	if cv_fold is None:
		cv_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
	
	random_search = RandomizedSearchCV(
		estimator=model,
		param_distributions=param_distributions,
		n_iter=n_iter,
		cv=cv_fold,
		scoring=scoring,
		random_state=random_state,
		n_jobs=-1,
		verbose=verbose
	)
	
	random_search.fit(X_train, y_train)
	return random_search.best_estimator_, random_search.best_params_, random_search.best_score_

def tune_model_grid(model, param_grid, X_train, y_train, 
					cv_fold=None, scoring='roc_auc', verbose=0):
	"""
	Use GridSearchCV to refine hyperparameter search in target region.
	
	Args:
		model: Base model to tune
		param_grid: Dictionary of parameter grid
		X_train: Training features
		y_train: Training target
		cv_fold: Pre-defined CV fold object (for fairness across models)
		scoring: Scoring metric
		verbose: Verbosity level
	
	Returns:
		Best model from GridSearchCV
	"""
	if cv_fold is None:
		cv_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
	
	grid_search = GridSearchCV(
		estimator=model,
		param_grid=param_grid,
		cv=cv_fold,
		scoring=scoring,
		n_jobs=-1,
		verbose=verbose
	)
	
	grid_search.fit(X_train, y_train)
	return grid_search.best_estimator_, grid_search.best_params_, grid_search.best_score_

def evaluate_model(model, X_train, y_train, X_test, y_test, model_name="Model", verbose=True):  # type: ignore
	"""
	Evaluate model using ROC-AUC and F1 score.
	
	Args:
		model: Trained model
		X_train: Training features
		y_train: Training target
		X_test: Test features
		y_test: Test target
		model_name: Name for logging
		verbose: If True, print results
	
	Returns:
		Dictionary with evaluation metrics
	"""
	# Predictions
	y_train_pred = model.predict(X_train)
	y_test_pred = model.predict(X_test)
	
	# Probabilities for ROC-AUC
	y_train_proba = model.predict_proba(X_train)[:, 1]
	y_test_proba = model.predict_proba(X_test)[:, 1]
	
	# Calculate metrics
	train_roc_auc = roc_auc_score(y_train, y_train_proba)
	test_roc_auc = roc_auc_score(y_test, y_test_proba)
	train_f1 = f1_score(y_train, y_train_pred)
	test_f1 = f1_score(y_test, y_test_pred)
	
	results = {
		'train_roc_auc': train_roc_auc,
		'test_roc_auc': test_roc_auc,
		'train_f1': train_f1,
		'test_f1': test_f1,
		'confusion_matrix': confusion_matrix(y_test, y_test_pred),
		'classification_report': classification_report(y_test, y_test_pred)
	}
	
	if verbose:
		print(f"\n{model_name} Evaluation:")
		print(f"Train ROC-AUC: {train_roc_auc:.4f}")
		print(f"Test ROC-AUC: {test_roc_auc:.4f}")
		print(f"Train F1: {train_f1:.4f}")
		print(f"Test F1: {test_f1:.4f}")
		print(f"\nConfusion Matrix:\n{results['confusion_matrix']}")
		print(f"\nClassification Report:\n{results['classification_report']}")
	
	return results

def get_hyperparameter_grids():
	"""
	Define hyperparameter grids for each model.
	
    # Parameter ranges based on scikit-learn docs defaults and common practice from Hastie et al. (2009)
	# Expanded ranges for randomized search, narrower grids for refinement

	Returns:
		Dictionary of parameter distributions and grids for each model
	"""
	grids = {
		'logistic_regression': {
			'randomized': {
				'C': np.logspace(-4, 4, 20),
				'solver': ['liblinear', 'saga'],
				'max_iter': [500, 1000, 2000]
			},
			'grid': {
				'C': np.logspace(-2, 2, 10),
				'solver': ['liblinear', 'saga']
			}
		},
		'random_forest': {
			'randomized': {
				'n_estimators': [50, 100, 200, 300],
				'max_depth': [None, 10, 20, 30],
				'min_samples_split': [2, 5, 10],
				'min_samples_leaf': [1, 2, 4],
				'max_features': ['sqrt', 'log2', None]
			},
			'grid': {
				'n_estimators': [100, 200],
				'max_depth': [10, 20, None],
				'min_samples_split': [2, 5],
				'max_features': ['sqrt', 'log2']
			}
		},
		'gradient_boosting': {
			'randomized': {
				'n_estimators': [50, 100, 200],
				'learning_rate': [0.01, 0.1, 0.2],
				'max_depth': [3, 5, 7],
				'min_samples_split': [2, 5],
				'min_samples_leaf': [1, 2],
				'subsample': [0.8, 0.9, 1.0]
			},
			'grid': {
				'n_estimators': [100, 200],
				'learning_rate': [0.05, 0.1],
				'max_depth': [3, 5],
				'subsample': [0.8, 0.9]
			}
		},
		'neural_network': {
			'randomized': {
				'hidden_layer_sizes': [(50,), (100,), (100, 50), (150, 100)],
				'alpha': [0.0001, 0.001, 0.01],
				'learning_rate_init': [0.001, 0.01],
				'max_iter': [300, 500]
			},
			'grid': {
				'hidden_layer_sizes': [(100,), (100, 50)],
				'alpha': [0.0001, 0.001],
				'learning_rate_init': [0.001, 0.01]
			}
		}
	}
	return grids

def train_and_tune_models(X_train, y_train, X_test, y_test,   # type: ignore
						  use_randomized=True, use_grid=True, 
						  n_iter_randomized=50, cv=5, random_state=42, verbose=True):
	"""
	Train and tune all models using RandomizedSearchCV and GridSearchCV.
	
	Args:
		X_train: Training features
		y_train: Training target
		X_test: Test features
		y_test: Test target
		use_randomized: If True, use RandomizedSearchCV first
		use_grid: If True, refine with GridSearchCV
		n_iter_randomized: Number of iterations for RandomizedSearchCV
		cv: Number of CV folds
		random_state: Random seed
		verbose: If True, print progress
	
	Returns:
		Dictionary of trained models and their results
	"""
	models = {}
	results = {}
	grids = get_hyperparameter_grids()
	total_start_time = time.time()
	
	cv_fold = StratifiedKFold(n_splits=cv, shuffle=True, random_state=random_state)
	
	class_imbalance = check_class_imbalance(y_train)
	if verbose:
		print(f"Class imbalance detected: {class_imbalance}")
	
	model_start_time = time.time()
	if verbose:
		print("\nTraining Logistic Regression...")
	lr = get_logistic_regression(class_imbalance=class_imbalance, random_state=random_state)
	
	if use_randomized:
		lr_best, lr_params, lr_score = tune_model_randomized(
			lr, grids['logistic_regression']['randomized'],
			X_train, y_train, n_iter=n_iter_randomized, cv_fold=cv_fold, random_state=random_state, verbose=0
		)
		if verbose:
			print(f"RandomizedSearchCV best score: {lr_score:.4f}")
			print(f"Best params: {lr_params}")
		
		if use_grid:
			# Refine with grid search around best params
			grid_params = grids['logistic_regression']['grid'].copy()
			if 'C' in lr_params:
				best_c = lr_params['C']
				# Narrow C range around best value
				grid_params['C'] = np.logspace(np.log10(best_c * 0.5), np.log10(best_c * 2), 5)
			
			lr_best, lr_params, lr_score = tune_model_grid(
				lr_best, grid_params, X_train, y_train, cv_fold=cv_fold, verbose=0
			)
			if verbose:
				print(f"GridSearchCV best score: {lr_score:.4f}")
				print(f"Refined params: {lr_params}")
	else:
		lr_best = lr
		lr_best.fit(X_train, y_train)
	
	models['logistic_regression'] = lr_best
	results['logistic_regression'] = evaluate_model(  # type: ignore
		lr_best, X_train, y_train, X_test, y_test, "Logistic Regression", verbose=verbose  # type: ignore
	)
	model_time = time.time() - model_start_time
	accuracy = accuracy_score(y_test, lr_best.predict(X_test))
	print(f"Logistic Regression finished, training time: {model_time:.2f}s, accuracy: {accuracy:.4f}")
	
	model_start_time = time.time()
	if verbose:
		print("\nTraining Random Forest...")
	rf = get_random_forest(random_state=random_state)
	
	if use_randomized:
		rf_best, rf_params, rf_score = tune_model_randomized(
			rf, grids['random_forest']['randomized'],
			X_train, y_train, n_iter=n_iter_randomized, cv_fold=cv_fold, random_state=random_state, verbose=0
		)
		if verbose:
			print(f"RandomizedSearchCV best score: {rf_score:.4f}")
			print(f"Best params: {rf_params}")
		
		if use_grid:
			rf_best, rf_params, rf_score = tune_model_grid(
				rf_best, grids['random_forest']['grid'], X_train, y_train, cv_fold=cv_fold, verbose=0
			)
			if verbose:
				print(f"GridSearchCV best score: {rf_score:.4f}")
				print(f"Refined params: {rf_params}")
	else:
		rf_best = rf
		rf_best.fit(X_train, y_train)
	
	models['random_forest'] = rf_best
	results['random_forest'] = evaluate_model(  # type: ignore
		rf_best, X_train, y_train, X_test, y_test, "Random Forest", verbose=verbose  # type: ignore
	)
	model_time = time.time() - model_start_time
	accuracy = accuracy_score(y_test, rf_best.predict(X_test))
	print(f"Random Forest finished, training time: {model_time:.2f}s, accuracy: {accuracy:.4f}")
	
	model_start_time = time.time()
	if verbose:
		print("\nTraining Gradient Boosting...")
	gb = get_gradient_boosting(random_state=random_state)
	
	if use_randomized:
		gb_best, gb_params, gb_score = tune_model_randomized(
			gb, grids['gradient_boosting']['randomized'],
			X_train, y_train, n_iter=n_iter_randomized, cv_fold=cv_fold, random_state=random_state, verbose=0
		)
		if verbose:
			print(f"RandomizedSearchCV best score: {gb_score:.4f}")
			print(f"Best params: {gb_params}")
		
		if use_grid:
			gb_best, gb_params, gb_score = tune_model_grid(
				gb_best, grids['gradient_boosting']['grid'], X_train, y_train, cv_fold=cv_fold, verbose=0
			)
			if verbose:
				print(f"GridSearchCV best score: {gb_score:.4f}")
				print(f"Refined params: {gb_params}")
	else:
		gb_best = gb
		gb_best.fit(X_train, y_train)
	
	models['gradient_boosting'] = gb_best
	results['gradient_boosting'] = evaluate_model(  # type: ignore
		gb_best, X_train, y_train, X_test, y_test, "Gradient Boosting", verbose=verbose  # type: ignore
	)
	model_time = time.time() - model_start_time
	accuracy = accuracy_score(y_test, gb_best.predict(X_test))
	print(f"Gradient Boosting finished, training time: {model_time:.2f}s, accuracy: {accuracy:.4f}")
	
	model_start_time = time.time()
	if verbose:
		print("\nTraining Neural Network...")
	nn = get_neural_network(random_state=random_state)
	
	if use_randomized:
		nn_best, nn_params, nn_score = tune_model_randomized(
			nn, grids['neural_network']['randomized'],
			X_train, y_train, n_iter=n_iter_randomized, cv_fold=cv_fold, random_state=random_state, verbose=0
		)
		if verbose:
			print(f"RandomizedSearchCV best score: {nn_score:.4f}")
			print(f"Best params: {nn_params}")
		
		if use_grid:
			nn_best, nn_params, nn_score = tune_model_grid(
				nn_best, grids['neural_network']['grid'], X_train, y_train, cv_fold=cv_fold, verbose=0
			)
			if verbose:
				print(f"GridSearchCV best score: {nn_score:.4f}")
				print(f"Refined params: {nn_params}")
	else:
		nn_best = nn
		nn_best.fit(X_train, y_train)
	
	models['neural_network'] = nn_best
	results['neural_network'] = evaluate_model(  # type: ignore
		nn_best, X_train, y_train, X_test, y_test, "Neural Network", verbose=verbose  # type: ignore
	)
	model_time = time.time() - model_start_time
	accuracy = accuracy_score(y_test, nn_best.predict(X_test))
	print(f"Neural Network finished, training time: {model_time:.2f}s, accuracy: {accuracy:.4f}")
	
	total_time = time.time() - total_start_time
	
	best_model_name = None
	best_accuracy = 0
	for model_name, model in models.items():
		model_accuracy = accuracy_score(y_test, model.predict(X_test))
		if model_accuracy > best_accuracy:
			best_accuracy = model_accuracy
			best_model_name = model_name
	
	model_name_display = {
		'logistic_regression': 'Logistic Regression',
		'random_forest': 'Random Forest',
		'gradient_boosting': 'Gradient Boosting',
		'neural_network': 'Neural Network'
	}
	
	print(f"\nTotal training time: {total_time:.2f}s, most accurate: {model_name_display.get(best_model_name, best_model_name)} ({best_accuracy:.4f})")
	
	return models, results

def create_stacked_model(models_dict, X_train, y_train, random_state=42, calibrate=False):  # type: ignore
	"""
	Create stacked ensemble model using Random Forest + Gradient Boosting + Logistic Regression.
	
	Args:
		models_dict: Dictionary containing trained models
		X_train: Training features
		y_train: Training target
		random_state: Random seed
		calibrate: If True, calibrate models for reliable probability outputs
	
	Returns:
		Stacked model (VotingClassifier)
	"""
	# Get base models
	rf = models_dict.get('random_forest')
	gb = models_dict.get('gradient_boosting')
	lr = models_dict.get('logistic_regression')
	
	if rf is None or gb is None or lr is None:
		raise ValueError("Need random_forest, gradient_boosting, and logistic_regression models")
	
	if calibrate:
		rf_calibrated = CalibratedClassifierCV(rf, method='isotonic', cv=3)
		gb_calibrated = CalibratedClassifierCV(gb, method='isotonic', cv=3)
		lr_calibrated = CalibratedClassifierCV(lr, method='isotonic', cv=3)
		
		rf_calibrated.fit(X_train, y_train)
		gb_calibrated.fit(X_train, y_train)
		lr_calibrated.fit(X_train, y_train)
		
		estimators = [
			('rf', rf_calibrated),
			('gb', gb_calibrated),
			('lr', lr_calibrated)
		]
	else:
		estimators = [
			('rf', rf),
			('gb', gb),
			('lr', lr)
		]
	
	# Create voting classifier (soft voting for probabilities)
	stacked_model = VotingClassifier(
		estimators=estimators,
		voting='soft',
		weights=None
	)
	
	stacked_model.fit(X_train, y_train)
	return stacked_model

def get_feature_importance(model, feature_names, method='permutation', X_test=None, y_test=None):
	"""
	Get feature importance using different methods.
	
	Args:
		model: Trained model
		feature_names: List of feature names
		method: 'permutation', 'l1', or 'tree' (for tree-based models)
		X_test: Test features (needed for permutation importance)
		y_test: Test target (needed for permutation importance)
	
	Returns:
		DataFrame with feature importances
	"""
	from sklearn.inspection import permutation_importance
	
	if method == 'permutation' and X_test is not None and y_test is not None:
		perm_importance = permutation_importance(
			model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
		)
		importances = perm_importance.importances_mean
		std = perm_importance.importances_std
		
		importance_df = pd.DataFrame({
			'feature': feature_names,
			'importance': importances,
			'std': std
		}).sort_values('importance', ascending=False)
		
	elif method == 'l1' and hasattr(model, 'coef_'):
		# L1 importance for linear models
		coef = np.abs(model.coef_[0])
		importance_df = pd.DataFrame({
			'feature': feature_names,
			'importance': coef
		}).sort_values('importance', ascending=False)
		
	elif method == 'tree' and hasattr(model, 'feature_importances_'):
		# Tree-based feature importance
		importance_df = pd.DataFrame({
			'feature': feature_names,
			'importance': model.feature_importances_
		}).sort_values('importance', ascending=False)
		
	else:
		raise ValueError(f"Method {method} not available for this model type")
	
	return importance_df

def drop_negligible_features(X_train, X_test, feature_importance_df, threshold=0.01):
	"""
	Drop features with importance below threshold to reduce overfitting.
	
	Args:
		X_train: Training features
		X_test: Test features
		feature_importance_df: DataFrame with feature importances
		threshold: Minimum importance threshold (relative to max)
	
	Returns:
		Filtered X_train, X_test, and list of dropped features
	"""
	max_importance = feature_importance_df['importance'].max()
	threshold_value = max_importance * threshold
	
	important_features = feature_importance_df[
		feature_importance_df['importance'] >= threshold_value
	]['feature'].tolist()
	
	dropped_features = [f for f in X_train.columns if f not in important_features]
	
	X_train_filtered = X_train[important_features]
	X_test_filtered = X_test[important_features]
	
	return X_train_filtered, X_test_filtered, dropped_features

## Analysis and Evaluation Functions

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
from scipy import stats
from sklearn.metrics import (
	roc_auc_score, precision_score, recall_score, f1_score,
	confusion_matrix, average_precision_score, brier_score_loss
)
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.base import clone

def comprehensive_evaluation(model, X_test, y_test, model_name="Model", 
							cost_false_positive=100, cost_false_negative=5000, verbose=True):
	"""
	Comprehensive evaluation of a model with all relevant metrics.
	
	Args:
		model: Trained model
		X_test: Test features
		y_test: Test target
		model_name: Name of the model for display
		cost_false_positive: Business cost of false positive (incorrectly predicting default)
		cost_false_negative: Business cost of false negative (missing a default)
		verbose: If True, print all metrics
	
	Returns:
		Dictionary containing all evaluation metrics
	"""
	y_pred = model.predict(X_test)
	y_proba = model.predict_proba(X_test)[:, 1]
	
	roc_auc = roc_auc_score(y_test, y_proba)
	precision = precision_score(y_test, y_pred)
	recall = recall_score(y_test, y_pred)
	f1 = f1_score(y_test, y_pred)
	pr_auc = average_precision_score(y_test, y_proba)
	brier_score = brier_score_loss(y_test, y_proba)
	cm = confusion_matrix(y_test, y_pred)
	
	tn, fp, fn, tp = cm.ravel()
	
	total_cost = (fp * cost_false_positive) + (fn * cost_false_negative)
	cost_per_prediction = total_cost / len(y_test)
	
	results = {
		'roc_auc': roc_auc,
		'precision': precision,
		'recall': recall,
		'f1': f1,
		'pr_auc': pr_auc,
		'brier_score': brier_score,
		'confusion_matrix': cm,
		'true_negatives': tn,
		'false_positives': fp,
		'false_negatives': fn,
		'true_positives': tp,
		'total_cost': total_cost,
		'cost_per_prediction': cost_per_prediction
	}
	
	if verbose:
		print(f"\n{model_name} Comprehensive Evaluation")
		
		print(f"\nROC-AUC: {roc_auc:.4f}")
		print("Overall discrimination ability")
		
		print(f"\nPrecision: {precision:.4f}")
		print(f"Recall: {recall:.4f}")
		print(f"F1 Score: {f1:.4f}")
		print("Important when default is minority class or false negatives are costly")
		
		print(f"\nPR-AUC: {pr_auc:.4f}")
		print("Precision-Recall AUC - better than ROC-AUC for class imbalance")
		
		print(f"\nConfusion Matrix:")
		print(f"                Predicted")
		print(f"              No Default  Default")
		print(f"Actual No Default    {tn:5d}     {fp:5d}")
		print(f"       Default       {fn:5d}     {tp:5d}")
		print(f"\nFalse Positives: {fp}")
		print(f"False Negatives: {fn}")
		
		print(f"\nBrier Score: {brier_score:.4f}")
		print("Lower is better - measures probabilistic prediction quality")
		
		print(f"\nCost Analysis:")
		print(f"Cost per False Positive: ${cost_false_positive:,.2f}")
		print(f"Cost per False Negative: ${cost_false_negative:,.2f}")
		print(f"Total False Positives: {fp}")
		print(f"Total False Negatives: {fn}")
		print(f"Total Business Cost: ${total_cost:,.2f}")
		print(f"Cost per Prediction: ${cost_per_prediction:,.2f}")
		print(f"\nPotential writeoff (if all defaults missed): ${fn * cost_false_negative:,.2f}")
		print(f"Potential payback (if all correctly identified): ${tp * cost_false_negative:,.2f}")
	
	return results

def predict_default_probability(model, LIMIT_BAL=None, SEX=None, EDUCATION=None, MARRIAGE=None, 
					   AGE=None, PAY_0=None, PAY_2=None, PAY_3=None, PAY_4=None, 
					   PAY_5=None, PAY_6=None, BILL_AMT1=None, BILL_AMT2=None, 
					   BILL_AMT3=None, BILL_AMT4=None, BILL_AMT5=None, BILL_AMT6=None,
					   PAY_AMT1=None, PAY_AMT2=None, PAY_AMT3=None, PAY_AMT4=None,
					   PAY_AMT5=None, PAY_AMT6=None, feature_names=None, scaler=None):
	"""
	Predict default probability for a hypothetical person.
	
	Args:
		model: Trained model
		LIMIT_BAL: Credit limit balance
		SEX: Sex (1=male, 2=female)
		EDUCATION: Education level (1=graduate, 2=university, 3=high school, 4=others)
		MARRIAGE: Marital status (1=married, 2=single, 3=others)
		AGE: Age in years
		PAY_0 through PAY_6: Payment status (-1=pay duly, 0=no consumption, 1+=months overdue)
		BILL_AMT1 through BILL_AMT6: Bill statement amounts
		PAY_AMT1 through PAY_AMT6: Previous payment amounts
		feature_names: List of feature names in the order expected by the model
		scaler: Scaler object (StandardScaler or RobustScaler) used during training. Required if models were trained on scaled data.
	
	Returns:
		Probability of default (0-1)
	"""
	if feature_names is None:
		raise ValueError("feature_names must be provided - use X_train.columns.tolist() from your training data")
	
	feature_dict = {
		'LIMIT_BAL': LIMIT_BAL,
		'SEX': SEX,
		'EDUCATION': EDUCATION,
		'MARRIAGE': MARRIAGE,
		'AGE': AGE,
		'PAY_0': PAY_0,
		'PAY_2': PAY_2,
		'PAY_3': PAY_3,
		'PAY_4': PAY_4,
		'PAY_5': PAY_5,
		'PAY_6': PAY_6,
		'BILL_AMT1': BILL_AMT1,
		'BILL_AMT2': BILL_AMT2,
		'BILL_AMT3': BILL_AMT3,
		'BILL_AMT4': BILL_AMT4,
		'BILL_AMT5': BILL_AMT5,
		'BILL_AMT6': BILL_AMT6,
		'PAY_AMT1': PAY_AMT1,
		'PAY_AMT2': PAY_AMT2,
		'PAY_AMT3': PAY_AMT3,
		'PAY_AMT4': PAY_AMT4,
		'PAY_AMT5': PAY_AMT5,
		'PAY_AMT6': PAY_AMT6
	}
	
	feature_values = []
	for feature in feature_names:
		if feature in feature_dict:
			value = feature_dict[feature]
			if value is None:
				raise ValueError(f"Missing required feature: {feature}")
			feature_values.append(value)
		else:
			if feature == 'default_payment_next_month':
				continue
			if feature.startswith('SEX_') or feature.startswith('EDUCATION_') or feature.startswith('MARRIAGE_'):
				base_feature = feature.split('_')[0]
				if base_feature == 'SEX' and SEX is not None:
					feature_values.append(1 if feature == f'SEX_{SEX}' else 0)
				elif base_feature == 'EDUCATION' and EDUCATION is not None:
					feature_values.append(1 if feature == f'EDUCATION_{EDUCATION}' else 0)
				elif base_feature == 'MARRIAGE' and MARRIAGE is not None:
					feature_values.append(1 if feature == f'MARRIAGE_{MARRIAGE}' else 0)
				else:
					feature_values.append(0)
			elif 'total_bills_paid_to_date' in feature:
				if PAY_AMT1 is not None:
					feature_values.append(sum([PAY_AMT1 or 0, PAY_AMT2 or 0, PAY_AMT3 or 0, 
											  PAY_AMT4 or 0, PAY_AMT5 or 0, PAY_AMT6 or 0]))
				else:
					feature_values.append(0)
			elif 'avg_bill' in feature:
				if BILL_AMT1 is not None:
					bills = [BILL_AMT1 or 0, BILL_AMT2 or 0, BILL_AMT3 or 0, 
							BILL_AMT4 or 0, BILL_AMT5 or 0, BILL_AMT6 or 0]
					feature_values.append(np.mean(bills))
				else:
					feature_values.append(0)
			elif 'current_outstanding' in feature:
				if BILL_AMT1 is not None:
					feature_values.append(sum([BILL_AMT1 or 0, BILL_AMT2 or 0, BILL_AMT3 or 0, 
											  BILL_AMT4 or 0, BILL_AMT5 or 0, BILL_AMT6 or 0]))
				else:
					feature_values.append(0)
			elif 'amortised_debt' in feature:
				if BILL_AMT1 is not None and PAY_AMT1 is not None:
					bills = sum([BILL_AMT1 or 0, BILL_AMT2 or 0, BILL_AMT3 or 0, 
								BILL_AMT4 or 0, BILL_AMT5 or 0, BILL_AMT6 or 0])
					payments = sum([PAY_AMT1 or 0, PAY_AMT2 or 0, PAY_AMT3 or 0, 
								   PAY_AMT4 or 0, PAY_AMT5 or 0, PAY_AMT6 or 0])
					feature_values.append(max(0, bills - payments))
				else:
					feature_values.append(0)
			elif 'avg_overdue_months' in feature:
				if PAY_0 is not None:
					pay_status = [PAY_0 or 0, PAY_2 or 0, PAY_3 or 0, PAY_4 or 0, PAY_5 or 0, PAY_6 or 0]
					feature_values.append(np.mean(pay_status))
				else:
					feature_values.append(0)
			elif 'max_overdue_months' in feature:
				if PAY_0 is not None:
					pay_status = [PAY_0 or 0, PAY_2 or 0, PAY_3 or 0, PAY_4 or 0, PAY_5 or 0, PAY_6 or 0]
					feature_values.append(max(pay_status))
				else:
					feature_values.append(0)
			elif 'months_overdue_count' in feature:
				if PAY_0 is not None:
					pay_status = [PAY_0 or 0, PAY_2 or 0, PAY_3 or 0, PAY_4 or 0, PAY_5 or 0, PAY_6 or 0]
					feature_values.append(sum(1 for p in pay_status if p > 0))
				else:
					feature_values.append(0)
			elif 'credit_utilisation' in feature:
				if LIMIT_BAL is not None and LIMIT_BAL > 0:
					if BILL_AMT1 is not None:
						bills = sum([BILL_AMT1 or 0, BILL_AMT2 or 0, BILL_AMT3 or 0, 
									BILL_AMT4 or 0, BILL_AMT5 or 0, BILL_AMT6 or 0])
						feature_values.append(min(1.0, bills / LIMIT_BAL))
					else:
						feature_values.append(0)
				else:
					feature_values.append(0)
			else:
				feature_values.append(0)
	
	feature_array = np.array([feature_values])
	
	# Apply scaling if scaler is provided (models were trained on scaled data)
	if scaler is not None:
		feature_array = scaler.transform(feature_array)
	
	probability = model.predict_proba(feature_array)[0, 1]
	
	return probability

def cross_validate_models(models_dict, X_train, y_train, cv=5, random_state=42, verbose=True):
	"""
	Cross-validate all models using the same stratified CV folds for fairness.
	Reports ROC-AUC and PR-AUC with mean ± std.
	
	Args:
		models_dict: Dictionary of trained models
		X_train: Training features
		y_train: Training target
		cv: Number of CV folds
		random_state: Random seed
		verbose: If True, print results
	
	Returns:
		Dictionary with CV results for each model
	"""
	cv_fold = StratifiedKFold(n_splits=cv, shuffle=True, random_state=random_state)
	results = {}
	
	if verbose:
		print("\nCross-Validation Evaluation (Same Folds for All Models)")
	
	for model_name, model in models_dict.items():
		roc_auc_scores = []
		pr_auc_scores = []
		
		model_clone = clone(model)
		
		for fold_idx, (train_idx, val_idx) in enumerate(cv_fold.split(X_train, y_train)):
			if isinstance(X_train, pd.DataFrame):
				X_fold_train = X_train.iloc[train_idx]
				X_fold_val = X_train.iloc[val_idx]
			else:
				X_fold_train = X_train[train_idx]
				X_fold_val = X_train[val_idx]
			
			if isinstance(y_train, pd.Series):
				y_fold_train = y_train.iloc[train_idx]
				y_fold_val = y_train.iloc[val_idx]
			else:
				y_fold_train = y_train[train_idx]
				y_fold_val = y_train[val_idx]
			
			model_clone.fit(X_fold_train, y_fold_train)
			y_proba = model_clone.predict_proba(X_fold_val)[:, 1]
			
			roc_auc = roc_auc_score(y_fold_val, y_proba)
			pr_auc = average_precision_score(y_fold_val, y_proba)
			
			roc_auc_scores.append(roc_auc)
			pr_auc_scores.append(pr_auc)
		
		roc_mean = np.mean(roc_auc_scores)
		roc_std = np.std(roc_auc_scores)
		pr_mean = np.mean(pr_auc_scores)
		pr_std = np.std(pr_auc_scores)
		
		results[model_name] = {
			'roc_auc_scores': roc_auc_scores,
			'pr_auc_scores': pr_auc_scores,
			'roc_auc_mean': roc_mean,
			'roc_auc_std': roc_std,
			'pr_auc_mean': pr_mean,
			'pr_auc_std': pr_std
		}
		
		if verbose:
			display_name = {
				'logistic_regression': 'Logistic Regression',
				'random_forest': 'Random Forest',
				'gradient_boosting': 'Gradient Boosting',
				'neural_network': 'Neural Network',
				'stacked_model': 'Stacked Ensemble'
			}.get(model_name, model_name)
			
			print(f"\n{display_name}:")
			print(f"  ROC-AUC: {roc_mean:.4f} ± {roc_std:.4f}")
			print(f"  PR-AUC:  {pr_mean:.4f} ± {pr_std:.4f}")
	
	return results

def mcnemar_test(y_true, y_pred1, y_pred2):
	"""
	McNemar's test for comparing two classifiers on the same dataset.
	
	Args:
		y_true: True labels
		y_pred1: Predictions from model 1
		y_pred2: Predictions from model 2
	
	Returns:
		chi2 statistic, p-value
	"""
	correct1 = (y_true == y_pred1)
	correct2 = (y_true == y_pred2)
	
	b = np.sum((correct1 == False) & (correct2 == True))
	c = np.sum((correct1 == True) & (correct2 == False))
	
	if b + c == 0:
		return 0.0, 1.0
	
	chi2 = ((abs(b - c) - 1) ** 2) / (b + c)
	p_value = 1 - stats.chi2.cdf(chi2, df=1)
	
	return chi2, p_value

def paired_t_test(scores1, scores2):
	"""
	Paired t-test for comparing two sets of scores (e.g., from CV folds).
	
	Args:
		scores1: Scores from model 1 (array-like)
		scores2: Scores from model 2 (array-like)
	
	Returns:
		t statistic, p-value
	"""
	scores1 = np.array(scores1)
	scores2 = np.array(scores2)
	
	differences = scores1 - scores2
	t_stat, p_value = stats.ttest_1samp(differences, 0)
	
	return t_stat, p_value

def statistical_comparison(models_dict, X_test, y_test, cv_results=None, verbose=True):
	"""
	Perform McNemar's test and paired t-test to compare models.
	
	Args:
		models_dict: Dictionary of trained models
		X_test: Test features
		y_test: Test target
		cv_results: Optional CV results from cross_validate_models
		verbose: If True, print results
	
	Returns:
		Dictionary with statistical test results
	"""
	if verbose:
		print("\nStatistical Model Comparison")
	
	model_names = list(models_dict.keys())
	predictions = {}
	
	for model_name, model in models_dict.items():
		predictions[model_name] = model.predict(X_test)
	
	comparison_results = {}
	
	for i, model1_name in enumerate(model_names):
		for model2_name in model_names[i+1:]:
			pred1 = predictions[model1_name]
			pred2 = predictions[model2_name]
			
			chi2, p_mcnemar = mcnemar_test(y_test, pred1, pred2)
			
			comparison_key = f"{model1_name}_vs_{model2_name}"
			comparison_results[comparison_key] = {
				'mcnemar_chi2': chi2,
				'mcnemar_p': p_mcnemar
			}
			
			if cv_results is not None:
				if model1_name in cv_results and model2_name in cv_results:
					t_stat, p_ttest = paired_t_test(
						cv_results[model1_name]['roc_auc_scores'],
						cv_results[model2_name]['roc_auc_scores']
					)
					comparison_results[comparison_key]['ttest_t'] = t_stat
					comparison_results[comparison_key]['ttest_p'] = p_ttest
			
			if verbose:
				display1 = {
					'logistic_regression': 'Logistic Regression',
					'random_forest': 'Random Forest',
					'gradient_boosting': 'Gradient Boosting',
					'neural_network': 'Neural Network'
				}.get(model1_name, model1_name)
				
				display2 = {
					'logistic_regression': 'Logistic Regression',
					'random_forest': 'Random Forest',
					'gradient_boosting': 'Gradient Boosting',
					'neural_network': 'Neural Network'
				}.get(model2_name, model2_name)
				
				print(f"\n{display1} vs {display2}:")
				print(f"  McNemar's test: chi2={chi2:.4f}, p={p_mcnemar:.4f}")
				if 'ttest_t' in comparison_results[comparison_key]:
					print(f"  Paired t-test: t={comparison_results[comparison_key]['ttest_t']:.4f}, p={comparison_results[comparison_key]['ttest_p']:.4f}")
	
	return comparison_results

def plot_feature_importance_and_shap(model, X_train, X_test, feature_names, model_name="Model", 
							top_n=20, save_plots=False, y_test=None):
	"""
	Plot feature importance and SHAP values for model interpretation.
	
	Args:
		model: Trained model
		X_train: Training features
		X_test: Test features (for SHAP)
		feature_names: List of feature names
		model_name: Name of model for display
		top_n: Number of top features to show
		save_plots: If True, save plots to files
		y_test: Test target (needed for permutation importance)
	"""
	import os
	
	# Create results folder if saving plots
	if save_plots:
		os.makedirs('results', exist_ok=True)
	
	if hasattr(model, 'feature_importances_'):
		method = 'tree'
		importance_df = get_feature_importance(model, feature_names, method=method)
	else:
		if y_test is not None:
			method = 'permutation'
			importance_df = get_feature_importance(model, feature_names, method=method, X_test=X_test, y_test=y_test)
		else:
			print(f"Warning: y_test not provided, cannot compute permutation importance for {model_name}")
			return
	
	top_features = importance_df.head(top_n)
	
	plt.figure(figsize=(10, 8))
	plt.barh(range(len(top_features)), top_features['importance'].values)
	plt.yticks(range(len(top_features)), top_features['feature'].values)
	plt.xlabel('Importance')
	plt.title(f'{model_name} - Top {top_n} Feature Importances')
	plt.gca().invert_yaxis()
	plt.tight_layout()
	
	if save_plots:
		plt.savefig(f'results/{model_name}_feature_importance.png', dpi=150)
		print(f"Saved feature importance plot to results/{model_name}_feature_importance.png")
	else:
		plt.show()
	
	plt.close()
	
	try:
		X_test_sample = X_test[:100] if hasattr(X_test, 'iloc') else X_test[:100]
		X_train_sample = X_train[:100] if hasattr(X_train, 'iloc') else X_train[:100]
		
		if hasattr(model, 'feature_importances_'):
			explainer = shap.TreeExplainer(model)
			shap_values = explainer.shap_values(X_test_sample)
		elif hasattr(model, 'coef_'):
			explainer = shap.LinearExplainer(model, X_train_sample)
			shap_values = explainer.shap_values(X_test_sample)
		else:
			def model_predict(X):
				return model.predict_proba(X)[:, 1]
			explainer = shap.KernelExplainer(model_predict, X_train_sample)
			shap_values = explainer.shap_values(X_test_sample)
		
		# Handle different SHAP value formats
		if isinstance(shap_values, list):
			if len(shap_values) == 2:
				# Binary classification: use positive class (index 1)
				shap_values = shap_values[1]
			else:
				shap_values = shap_values[0]
		
		shap_values = np.array(shap_values)
		
		# Handle 3D arrays (samples, classes, features) - take positive class
		if len(shap_values.shape) == 3:
			# For binary classification, shape is (n_samples, n_classes, n_features)
			# Take the positive class (index 1) or the last class
			if shap_values.shape[1] == 2:
				shap_values = shap_values[:, 1, :]  # Take positive class
			else:
				shap_values = shap_values[:, -1, :]  # Take last class
		
		# Ensure 2D array (samples, features)
		if len(shap_values.shape) == 1:
			shap_values = shap_values.reshape(1, -1)
		
		# Convert X_test_sample to numpy array and ensure shapes match
		if hasattr(X_test_sample, 'values'):
			X_test_sample_array = X_test_sample.values
		elif hasattr(X_test_sample, 'to_numpy'):
			X_test_sample_array = X_test_sample.to_numpy()
		else:
			X_test_sample_array = np.array(X_test_sample)
		
		# Ensure both are 2D
		if len(X_test_sample_array.shape) == 1:
			X_test_sample_array = X_test_sample_array.reshape(-1, 1)
		
		# Ensure both have the same number of samples (should match, but be safe)
		if shap_values.shape[0] != X_test_sample_array.shape[0]:
			min_samples = min(shap_values.shape[0], X_test_sample_array.shape[0])
			shap_values = shap_values[:min_samples, :]
			X_test_sample_array = X_test_sample_array[:min_samples, :]
		
		# Ensure both have the same number of features
		if shap_values.shape[1] != X_test_sample_array.shape[1]:
			min_features = min(shap_values.shape[1], X_test_sample_array.shape[1])
			shap_values = shap_values[:, :min_features]
			X_test_sample_array = X_test_sample_array[:, :min_features]
			feature_names_plot = feature_names[:min_features]
		else:
			feature_names_plot = feature_names
		
		plt.figure(figsize=(10, 8))
		shap.summary_plot(shap_values, X_test_sample_array, feature_names=feature_names_plot, show=False, max_display=top_n)
		plt.title(f'{model_name} - SHAP Summary Plot')
		plt.tight_layout()
		
		if save_plots:
			plt.savefig(f'results/{model_name}_shap_summary.png', dpi=150, bbox_inches='tight')
			print(f"Saved SHAP summary plot to results/{model_name}_shap_summary.png")
		else:
			plt.show()
		
		plt.close()
		
		# SHAP Bar Plot - use already processed shap_values
		try:
			# shap_values is already processed to 2D array (samples, features)
			# Calculate mean absolute SHAP values per feature
			mean_shap = np.abs(shap_values).mean(axis=0)  # Mean across samples
			
			# Ensure feature names match
			if len(mean_shap) != len(feature_names):
				min_len = min(len(mean_shap), len(feature_names))
				mean_shap = mean_shap[:min_len]
				feature_names_use = feature_names[:min_len]
			else:
				feature_names_use = feature_names
			
			# Get top N features
			top_indices = np.argsort(mean_shap)[::-1][:top_n]
			top_mean_shap = mean_shap[top_indices]
			top_feature_names = [feature_names_use[i] for i in top_indices]
			
			# Create bar plot manually
			plt.figure(figsize=(10, 8))
			plt.barh(range(len(top_indices)), top_mean_shap)
			plt.yticks(range(len(top_indices)), top_feature_names)
			plt.xlabel('Mean |SHAP value|')
			plt.title(f'{model_name} - SHAP Bar Plot')
			plt.gca().invert_yaxis()
			plt.tight_layout()
			
			if save_plots:
				plt.savefig(f'results/{model_name}_shap_bar.png', dpi=150, bbox_inches='tight')
				print(f"Saved SHAP bar plot to results/{model_name}_shap_bar.png")
			else:
				plt.show()
			
			plt.close()
		except Exception as bar_error:
			print(f"Warning: Could not generate SHAP bar plot: {bar_error}")
		
	except Exception as e:
		print(f"Error generating SHAP plots: {e}")

## Execute Pipeline

In [14]:
# Load and preprocess data
X_train, X_test, y_train, y_test, scaler = preprocess_data(
    'default_of_credit_card_clients.xls',
    verbose=True,
    split_data=True,
    apply_scaling=True,
    random_state=42
)

Renamed columns:
  X1 -> LIMIT_BAL
  X2 -> SEX
  X3 -> EDUCATION
  X4 -> MARRIAGE
  X5 -> AGE
  X6 -> PAY_0
  X7 -> PAY_2
  X8 -> PAY_3
  X9 -> PAY_4
  X10 -> PAY_5
  X11 -> PAY_6
  X12 -> BILL_AMT1
  X13 -> BILL_AMT2
  X14 -> BILL_AMT3
  X15 -> BILL_AMT4
  X16 -> BILL_AMT5
  X17 -> BILL_AMT6
  X18 -> PAY_AMT1
  X19 -> PAY_AMT2
  X20 -> PAY_AMT3
  X21 -> PAY_AMT4
  X22 -> PAY_AMT5
  X23 -> PAY_AMT6
  Y -> default_payment_next_month

INITIAL DATA EXPLORATION

DataFrame head:
  Unnamed: 0  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  \
0         ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3   
1          1      20000    2          2         1   24      2      2     -1   
2          2     120000    2          2         2   26     -1      2      0   
3          3      90000    2          2         2   34      0      0      0   
4          4      50000    2          2         1   37      0      0      0   

   PAY_4  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6

In [15]:
# Train and tune models
models, results = train_and_tune_models(
    X_train, y_train, X_test, y_test,
    use_randomized=True,
    use_grid=True,
    n_iter_randomized=20,
    cv=5,
    random_state=42,
    verbose=True
)

Class imbalance detected: True

Training Logistic Regression...
RandomizedSearchCV best score: 0.7629
Best params: {'solver': 'liblinear', 'max_iter': 1000, 'C': np.float64(1438.44988828766)}
GridSearchCV best score: 0.7629
Refined params: {'C': np.float64(2876.8997765753206), 'solver': 'liblinear'}

Logistic Regression Evaluation:
Train ROC-AUC: 0.7666
Test ROC-AUC: 0.7484
Train F1: 0.5268
Test F1: 0.5094

Confusion Matrix:
[[3672 1002]
 [ 531  796]]

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.79      0.83      4674
           1       0.44      0.60      0.51      1327

    accuracy                           0.74      6001
   macro avg       0.66      0.69      0.67      6001
weighted avg       0.78      0.74      0.76      6001

Logistic Regression finished, training time: 78.90s, accuracy: 0.7445

Training Random Forest...
RandomizedSearchCV best score: 0.7823
Best params: {'n_estimators': 200, 'min_samples_split': 10

In [16]:
print("\n Creating stacked model")
stacked_model = create_stacked_model(models, X_train, y_train, random_state=42, calibrate=False)

# Add stacked model to models dictionary
models['stacked_model'] = stacked_model

# Evaluate stacked model
print("\nEvaluating Stacked Model...")
stacked_results = evaluate_model( 
    stacked_model, X_train, y_train, X_test, y_test, "Stacked Ensemble", verbose=True
)

# Add to results dictionary
results['stacked_model'] = stacked_results

stacked_accuracy = accuracy_score(y_test, stacked_model.predict(X_test))
print(f"\nStacked Ensemble finished, accuracy: {stacked_accuracy:.4f}")



 Creating stacked model

Evaluating Stacked Model...

Stacked Ensemble Evaluation:
Train ROC-AUC: 0.8292
Test ROC-AUC: 0.7739
Train F1: 0.5733
Test F1: 0.5112

Confusion Matrix:
[[4274  400]
 [ 734  593]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.91      0.88      4674
           1       0.60      0.45      0.51      1327

    accuracy                           0.81      6001
   macro avg       0.73      0.68      0.70      6001
weighted avg       0.80      0.81      0.80      6001


Stacked Ensemble finished, accuracy: 0.8110


In [17]:
# Cross-validation evaluation
cv_results = cross_validate_models(models, X_train, y_train, cv=5, random_state=42, verbose=True)


Cross-Validation Evaluation (Same Folds for All Models)

Logistic Regression:
  ROC-AUC: 0.7629 ± 0.0064
  PR-AUC:  0.5176 ± 0.0100

Random Forest:
  ROC-AUC: 0.7822 ± 0.0052
  PR-AUC:  0.5617 ± 0.0100

Gradient Boosting:
  ROC-AUC: 0.7832 ± 0.0055
  PR-AUC:  0.5615 ± 0.0085

Neural Network:
  ROC-AUC: 0.7751 ± 0.0082
  PR-AUC:  0.5445 ± 0.0134

Stacked Ensemble:
  ROC-AUC: 0.7829 ± 0.0053
  PR-AUC:  0.5624 ± 0.0077


In [18]:
# Statistical model comparison
stat_results = statistical_comparison(models, X_test, y_test, cv_results=cv_results, verbose=True)


Statistical Model Comparison

Logistic Regression vs Random Forest:
  McNemar's test: chi2=168.4831, p=0.0000
  Paired t-test: t=-8.2496, p=0.0012

Logistic Regression vs Gradient Boosting:
  McNemar's test: chi2=163.4013, p=0.0000
  Paired t-test: t=-8.9105, p=0.0009

Logistic Regression vs Neural Network:
  McNemar's test: chi2=158.3253, p=0.0000
  Paired t-test: t=-5.1306, p=0.0068

Logistic Regression vs stacked_model:
  McNemar's test: chi2=195.3194, p=0.0000
  Paired t-test: t=-13.4064, p=0.0002

Random Forest vs Gradient Boosting:
  McNemar's test: chi2=0.4623, p=0.4966
  Paired t-test: t=-0.9865, p=0.3797

Random Forest vs Neural Network:
  McNemar's test: chi2=1.9527, p=0.1623
  Paired t-test: t=2.8253, p=0.0476

Random Forest vs stacked_model:
  McNemar's test: chi2=2.2588, p=0.1329
  Paired t-test: t=-0.8038, p=0.4666

Gradient Boosting vs Neural Network:
  McNemar's test: chi2=0.5192, p=0.4712
  Paired t-test: t=2.7460, p=0.0516

Gradient Boosting vs stacked_model:
  McNem

In [19]:
# Comprehensive evaluation
for model_name, model in models.items():
    display_name = {
        'logistic_regression': 'Logistic Regression',
        'random_forest': 'Random Forest',
        'gradient_boosting': 'Gradient Boosting',
        'neural_network': 'Neural Network',
        'stacked_model': 'Stacked Ensemble'
    }.get(model_name, model_name)
    
    comprehensive_evaluation(
        model, X_test, y_test, 
        model_name=display_name,
        cost_false_positive=100,
        cost_false_negative=5000,
        verbose=True
    )


Logistic Regression Comprehensive Evaluation

ROC-AUC: 0.7484
Overall discrimination ability

Precision: 0.4427
Recall: 0.5998
F1 Score: 0.5094
Important when default is minority class or false negatives are costly

PR-AUC: 0.5056
Precision-Recall AUC - better than ROC-AUC for class imbalance

Confusion Matrix:
                Predicted
              No Default  Default
Actual No Default     3672      1002
       Default         531       796

False Positives: 1002
False Negatives: 531

Brier Score: 0.1944
Lower is better - measures probabilistic prediction quality

Cost Analysis:
Cost per False Positive: $100.00
Cost per False Negative: $5,000.00
Total False Positives: 1002
Total False Negatives: 531
Total Business Cost: $2,755,200.00
Cost per Prediction: $459.12

Potential writeoff (if all defaults missed): $2,655,000.00
Potential payback (if all correctly identified): $3,980,000.00

Random Forest Comprehensive Evaluation

ROC-AUC: 0.7742
Overall discrimination ability

Precision: 0

In [20]:
# Feature importance and SHAP analysis
for model_name, model in models.items():
    display_name = {
        'logistic_regression': 'Logistic Regression',
        'random_forest': 'Random Forest',
        'gradient_boosting': 'Gradient Boosting',
        'neural_network': 'Neural Network',
        'stacked_model': 'Stacked Ensemble'
    }.get(model_name, model_name)
    
    print(f"\nAnalyzing {display_name}...")
    plot_feature_importance_and_shap(
        model, X_train, X_test, X_train.columns.tolist(),
        model_name=display_name, top_n=20, save_plots=True, y_test=y_test
    )


Analyzing Logistic Regression...
Saved feature importance plot to results/Logistic Regression_feature_importance.png
Saved SHAP summary plot to results/Logistic Regression_shap_summary.png
Saved SHAP bar plot to results/Logistic Regression_shap_bar.png

Analyzing Random Forest...
Saved feature importance plot to results/Random Forest_feature_importance.png
Saved SHAP summary plot to results/Random Forest_shap_summary.png
Saved SHAP bar plot to results/Random Forest_shap_bar.png

Analyzing Gradient Boosting...
Saved feature importance plot to results/Gradient Boosting_feature_importance.png
Saved SHAP summary plot to results/Gradient Boosting_shap_summary.png
Saved SHAP bar plot to results/Gradient Boosting_shap_bar.png

Analyzing Neural Network...
Saved feature importance plot to results/Neural Network_feature_importance.png


100%|██████████| 100/100 [00:32<00:00,  3.08it/s]


Saved SHAP summary plot to results/Neural Network_shap_summary.png
Saved SHAP bar plot to results/Neural Network_shap_bar.png

Analyzing Stacked Ensemble...
Saved feature importance plot to results/Stacked Ensemble_feature_importance.png


100%|██████████| 100/100 [01:12<00:00,  1.37it/s]


Saved SHAP summary plot to results/Stacked Ensemble_shap_summary.png
Saved SHAP bar plot to results/Stacked Ensemble_shap_bar.png


In [21]:
print("EXAMPLE PREDICTIONS")

test_cases = [
    {
        'name': 'Case 1: Low Risk Customer',
        'LIMIT_BAL': 20000, 'SEX': 2, 'EDUCATION': 2, 'MARRIAGE': 1, 'AGE': 30,
        'PAY_0': 0, 'PAY_2': 0, 'PAY_3': 0, 'PAY_4': 0, 'PAY_5': 0, 'PAY_6': 0,
        'BILL_AMT1': 5000, 'BILL_AMT2': 4500, 'BILL_AMT3': 4000, 'BILL_AMT4': 3500, 'BILL_AMT5': 3000, 'BILL_AMT6': 2500,
        'PAY_AMT1': 2000, 'PAY_AMT2': 2000, 'PAY_AMT3': 2000, 'PAY_AMT4': 2000, 'PAY_AMT5': 2000, 'PAY_AMT6': 2000
    },
    {
        'name': 'Case 2: Medium Risk Customer',
        'LIMIT_BAL': 15000, 'SEX': 1, 'EDUCATION': 3, 'MARRIAGE': 2, 'AGE': 35,
        'PAY_0': 1, 'PAY_2': 0, 'PAY_3': 1, 'PAY_4': 0, 'PAY_5': 0, 'PAY_6': 0,
        'BILL_AMT1': 12000, 'BILL_AMT2': 11000, 'BILL_AMT3': 10000, 'BILL_AMT4': 9000, 'BILL_AMT5': 8000, 'BILL_AMT6': 7000,
        'PAY_AMT1': 3000, 'PAY_AMT2': 2500, 'PAY_AMT3': 2000, 'PAY_AMT4': 2000, 'PAY_AMT5': 1500, 'PAY_AMT6': 1500
    },
    {
        'name': 'Case 3: High Risk Customer',
        'LIMIT_BAL': 10000, 'SEX': 1, 'EDUCATION': 4, 'MARRIAGE': 3, 'AGE': 45,
        'PAY_0': 2, 'PAY_2': 2, 'PAY_3': 1, 'PAY_4': 1, 'PAY_5': 0, 'PAY_6': 0,
        'BILL_AMT1': 9500, 'BILL_AMT2': 9000, 'BILL_AMT3': 8500, 'BILL_AMT4': 8000, 'BILL_AMT5': 7500, 'BILL_AMT6': 7000,
        'PAY_AMT1': 500, 'PAY_AMT2': 500, 'PAY_AMT3': 1000, 'PAY_AMT4': 1000, 'PAY_AMT5': 500, 'PAY_AMT6': 500
    },
    {
        'name': 'Case 4: Very High Risk Customer',
        'LIMIT_BAL': 50000, 'SEX': 2, 'EDUCATION': 1, 'MARRIAGE': 1, 'AGE': 28,
        'PAY_0': 3, 'PAY_2': 3, 'PAY_3': 2, 'PAY_4': 2, 'PAY_5': 1, 'PAY_6': 1,
        'BILL_AMT1': 48000, 'BILL_AMT2': 47000, 'BILL_AMT3': 46000, 'BILL_AMT4': 45000, 'BILL_AMT5': 44000, 'BILL_AMT6': 43000,
        'PAY_AMT1': 1000, 'PAY_AMT2': 1000, 'PAY_AMT3': 1000, 'PAY_AMT4': 1000, 'PAY_AMT5': 1000, 'PAY_AMT6': 1000
    },
    {
        'name': 'Case 5: Recovering Customer',
        'LIMIT_BAL': 30000, 'SEX': 1, 'EDUCATION': 2, 'MARRIAGE': 1, 'AGE': 40,
        'PAY_0': 0, 'PAY_2': 0, 'PAY_3': 1, 'PAY_4': 1, 'PAY_5': 2, 'PAY_6': 2,
        'BILL_AMT1': 8000, 'BILL_AMT2': 10000, 'BILL_AMT3': 12000, 'BILL_AMT4': 14000, 'BILL_AMT5': 16000, 'BILL_AMT6': 18000,
        'PAY_AMT1': 5000, 'PAY_AMT2': 5000, 'PAY_AMT3': 5000, 'PAY_AMT4': 5000, 'PAY_AMT5': 5000, 'PAY_AMT6': 5000
    }
]

feature_names_list = X_train.columns.tolist()

for case in test_cases:
    case_name = case.pop('name')
    print(f"\n{case_name}")

    probabilities = {}
    for model_name, model in models.items():
        display_name = {
            'logistic_regression': 'Logistic Regression',
            'random_forest': 'Random Forest',
            'gradient_boosting': 'Gradient Boosting',
            'neural_network': 'Neural Network',
            'stacked_model': 'Stacked Ensemble'
        }.get(model_name, model_name)

        prob_default = predict_default_probability(
            model,
            feature_names=feature_names_list,
            scaler=scaler,
            **case
        )
        probabilities[display_name] = prob_default
        prob_payback = 1 - prob_default
        print(f"{display_name}: Default={prob_default:.4f} ({prob_default*100:.2f}%), Payback={prob_payback:.4f} ({prob_payback*100:.2f}%)")

    avg_prob_default = np.mean(list(probabilities.values()))
    avg_prob_payback = 1 - avg_prob_default
    
    print(f"\nAverage across all models:")
    print(f"  Probability of DEFAULT: {avg_prob_default:.4f} ({avg_prob_default*100:.2f}%)")
    print(f"  Probability of PAYING BACK: {avg_prob_payback:.4f} ({avg_prob_payback*100:.2f}%)")
    
    if avg_prob_default >= 0.5:
        print(f"  Prediction: WILL DEFAULT (risk threshold: 50%)")
    else:
        print(f"  Prediction: WILL PAY BACK (risk threshold: 50%)")

EXAMPLE PREDICTIONS

Case 1: Low Risk Customer
Logistic Regression: Default=0.3583 (35.83%), Payback=0.6417 (64.17%)
Random Forest: Default=0.1361 (13.61%), Payback=0.8639 (86.39%)
Gradient Boosting: Default=0.1513 (15.13%), Payback=0.8487 (84.87%)
Neural Network: Default=0.1506 (15.06%), Payback=0.8494 (84.94%)
Stacked Ensemble: Default=0.2152 (21.52%), Payback=0.7848 (78.48%)

Average across all models:
  Probability of DEFAULT: 0.2023 (20.23%)
  Probability of PAYING BACK: 0.7977 (79.77%)
  Prediction: WILL PAY BACK (risk threshold: 50%)

Case 2: Medium Risk Customer
Logistic Regression: Default=0.6543 (65.43%), Payback=0.3457 (34.57%)
Random Forest: Default=0.2454 (24.54%), Payback=0.7546 (75.46%)
Gradient Boosting: Default=0.2104 (21.04%), Payback=0.7896 (78.96%)
Neural Network: Default=0.4065 (40.65%), Payback=0.5935 (59.35%)
Stacked Ensemble: Default=0.3701 (37.01%), Payback=0.6299 (62.99%)

Average across all models:
  Probability of DEFAULT: 0.3773 (37.73%)
  Probability of PA